In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import pandas as pd
import numpy as np
import torch

# Charger les données
df = pd.read_csv('top_1000_cleaned_reviews.csv', encoding='utf-8-sig', sep=';')


# Charger le tokenizer et le modèle
model_name = "cardiffnlp/xlm-roberta-base-tweet-sentiment-fr"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Fonction pour tokeniser les données en lots
def batch_tokenize(texts, batch_size=16):
    # Tokenisation en lots pour éviter les problèmes de mémoire
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
    return inputs

# Liste pour stocker les prédictions
predictions = []

# Traitement par lots
batch_size = 16
for i in range(0, len(df), batch_size):
    # Assurez-vous que vous utilisez la bonne colonne
    batch_reviews = df['content'][i:i+batch_size].tolist()  # Obtenez le lot suivant d'avis
    if len(batch_reviews) == 0:
        continue  # Passez au lot suivant s'il est vide
    inputs = batch_tokenize(batch_reviews, batch_size=batch_size)
        
    # Prédictions du modèle pour le lot
    with torch.no_grad():  # Nous n'avons pas besoin de gradients pour l'inférence
        outputs = model(**inputs)
    
    logits = outputs.logits.detach().numpy()  # Extraire les logits (scores de prédiction bruts)
    
    # Obtenez les prédictions de sentiment pour chaque avis
    for logit in logits:
        sentiment = model.config.id2label[np.argmax(softmax(logit))]
        predictions.append(sentiment)

# Ajouter les sentiments prédits au DataFrame
df['Sentiment'] = predictions

# Résumé des sentiments
sentiment_summary = df['Sentiment'].value_counts()

# Afficher les résultats sous forme de pourcentages
total_reviews = len(df)
sentiment_summary_percent = sentiment_summary / total_reviews * 100
print("\nRésumé des sentiments (en %):")
for sentiment, percentage in sentiment_summary_percent.items():
    print(f"  {sentiment.capitalize()}: {np.round(percentage, 2)}%")
    
# Sauvegarder le DataFrame avec les sentiments dans un fichier CSV
df.to_csv('reviews_with_sentiments.csv', encoding='utf-8-sig', sep=';', index=False)

print("\nLes sentiments ont été ajoutés.")

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

# Vérifier et télécharger les stopwords si nécessaire
try:
    french_stopwords = stopwords.words('french')
except LookupError:
    nltk.download('stopwords')
    # Charger les stop words en français
    french_stopwords = stopwords.words('french')

# Fonction pour extraire des bigrammes/trigrammes avec une liste de stop words personnalisée
def extract_ngrams(reviews, ngram_range=(2, 2), n=10):
    vectorizer = CountVectorizer(ngram_range=ngram_range, stop_words=french_stopwords, max_features=n)
    X = vectorizer.fit_transform(reviews)
    ngrams = vectorizer.get_feature_names_out()
    return ngrams

# Exemple d'utilisation pour les bigrammes
positive_reviews = df[df['Sentiment'] == "positive"]['content']
negative_reviews = df[df['Sentiment'] == "negative"]['content']

# Extraire des trigrammes
positive_trigrams = extract_ngrams(positive_reviews, ngram_range=(3, 3), n=10)
negative_trigrams = extract_ngrams(negative_reviews, ngram_range=(3, 3), n=10)

print("Top 10 Trigrams in Positive Reviews:", positive_trigrams)
print("Top 10 Trigrams in Negative Reviews:", negative_trigrams)


Top 10 Trigrams in Positive Reviews: ['application écouter musique' 'beaucoup cette application'
 'depuis dernière mise' 'depuis plusieurs années' 'dernière mise jour'
 'jadore cette application' 'jaime beaucoup cette' 'nouvelle mise jour'
 'permet découter musique' 'très bonne application']
Top 10 Trigrams in Negative Reviews: ['depuis dernière mise' 'depuis mise jour' 'depuis quelques jours'
 'depuis quelques mois' 'depuis quelques semaines' 'depuis quelques temps'
 'dernière mise jour' 'peut plus écouter' 'peux plus écouter'
 'très bonne application']
